# Módulos

> Index

> 1. Installing components

> 2. Super Class - DataColetctor

> 2. 1. Subclass - G1

> 2. 2. Subclass - Estadão

> 2. 3. Subclass - Veja





**1. Instalação de componentes**

In [ ]:
!pip install --upgrade google-cloud-language
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**2. Super Class - DataColetctor**


In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup as soup
from time import sleep, time
import pandas as pd
import joblib
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from datetime import datetime, timedelta
from sklearn.feature_extraction.text import TfidfVectorizer


class DataColector():
    def __init__(self):
        self.options = webdriver.ChromeOptions()
        self.options.add_argument('--headless')
        self.options.add_argument('--no-sandbox')
        self.options.add_argument("start-maximized");
        self.options.add_argument("disable-infobars");
        self.options.add_argument('--disable-dev-shm-usage')
        self.chrome = webdriver.Chrome('chromedriver', options=self.options)
    
    def acessa(self, url):
        self.chrome.get(url)
    
    def coleta_html(self):
        html = soup(self.chrome.page_source, 'html.parser')
        return html

    def scroll(self):
            scroll_pause_time = 3
            # Get scroll height
            last_height = self.chrome.execute_script("return document.body.scrollHeight")
            while True:
                # Scroll down to bottom
                self.chrome.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                # Wait to load page
                sleep(scroll_pause_time)
                # Calculate new scroll height and compare with last scroll height
                new_height = self.chrome.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height    

    def aguardar_por(self, xpath):
        WebDriverWait(self.chrome, 30).until(
            EC.visibility_of_element_located(
                (By.XPATH, xpath)))
        WebDriverWait(self.chrome, 30).until(
            EC.element_to_be_clickable(
                (By.XPATH, xpath)))

    def model(self):
        model = joblib.load('/content/drive/My Drive/Colab Notebooks/model_nlp.pyl')
        return model

    def tfidf_vectorize(self):
        tfidf = joblib.load('/content/drive/My Drive/Colab Notebooks/vectorize_tfidf.pyl')
        return tfidf

    def sair(self):
        self.chrome.quit()

if __name__ == '__main__':
    browser = DataColector()
    browser.sair()

**2.1 Subclass - G1**

In [4]:
class G1(DataColector):

    def clica_ver_mais(self, quantas_vezes):
        try:
            for n in range(1, quantas_vezes + 1):
                ver_mais = self.chrome.find_element_by_xpath('//*[@id="feed-placeholder"]/div/div/div[3]/a')
                self.chrome.execute_script(f'window.scrollTo(0,{n * 250});')
                self.chrome.execute_script('arguments[0].click();', ver_mais)
                sleep(1)
#               
        except Exception as e:
            print(f'Erro ao clicar em veja mais:', e)

    def coleta_noticia(self, fonte):
        return fonte.findAll('div', {'class': 'feed-post-body'})

    def coleta_resumo(self, resumo_em_text, resumo_em_list):
        try:
            if resumo_em_list:
                lista_com_resumos = []
                for r in resumo_em_list:
                    lista_com_resumos.append(r.text)
                restemp = '\n'.join(lista_com_resumos)
                return restemp

            if resumo_em_text:
                restemp = resumo_em_text[0].text
                return restemp

            else:
                pass
        except Exception as e:
            print(f'Erro ao coletar resumo:', e)

    def coleta_sessao_noticia(self, noticia):
        try:
            sessao = noticia.find('span', {'class': 'feed-post-metadata-section'})
            if sessao == None:
                return None
            return sessao.text
        except Exception as e:
            print(f'Erro ao coletar sessao:', e)

    @staticmethod
    def formatar_data(date_post):
        '''A data da noticícia é exibida pela diferença
        de tempo da postagem até o momento da exibição.
        Ex: Há 5 horas
        A função irá formatar a informação para o formato 
        de data completa aa/dd/mm hh:mm'''
        GMT_3 = 3
        data_br = datetime.now() - timedelta(hours=GMT_3)  # Horário do Brasil
        
        tipo_de_tempo = ['segundo', 'minuto', 'hora', 'dia']
        funcao_timedelta = ['seconds', 'minutes', 'hours', 'days']
        data_post = ''
        for index, tempo in enumerate(tipo_de_tempo):
            transforma_em_data = G1.retorna_numero_para_data(tempo, 
                                                            date_post)
            if not transforma_em_data == '':
                tipo_de_tempo_definido = int(transforma_em_data)
                kwargs = {}
                kwargs[funcao_timedelta[index]] = tipo_de_tempo_definido
                data_post = data_br - timedelta(**kwargs)
        return data_post

    @staticmethod
    def retorna_numero_para_data(time_type, date_post):
        time = ''
        if time_type in date_post:
            for char in date_post:
                if char.isdigit():
                    time += char
        return time


    def listar_noticias(self, cod_noticias):
        lista_de_noticias = []
        try:
            for noticia in cod_noticias:
                content = {} 
                content['titulo'] = noticia.a.text
                resumo_l = noticia.findAll('a', {'class': 'gui-color-primary gui-color-hover feed-post-body-title bstn-relatedtext'})
                resumo = noticia.findAll('div', {'class': 'feed-post-body-resumo'})
                content['resumo'] = self.coleta_resumo(resumo, resumo_l)
                sessao_noticia = self.coleta_sessao_noticia(noticia)
                content['sessao'] = sessao_noticia
                data = noticia.find('span', {'class': 'feed-post-datetime'}).text
                content['data'] = G1.formatar_data(data)
                content['portal'] = 'G1'
                content['link'] = noticia.a['href']
                lista_de_noticias.append(content)
            return lista_de_noticias
        except Exception as e:
            print(f'Erro ao tentar listar noticias:', e)

if __name__ == '__main__':
    lista_noticias_g1 = []
    while len(lista_noticias_g1) < 10:
        browser = G1()
        browser.acessa('https://g1.globo.com/')
        browser.clica_ver_mais(20)  # Cada clique, +10 noticias são exibidas
        html = browser.coleta_html()
        browser.sair()
        codigo_de_noticias = browser.coleta_noticia(html)
        lista_noticias_g1 = browser.listar_noticias(codigo_de_noticias)
    #    sleep(5)  
    # else:
    #     for valor in lista_noticias:
    #         for chave, conteudo in valor.items():
    #             print(f'{chave}: {conteudo}')
    #         print('-=' * 40)
    print(len(lista_noticias_g1))

186


**2.2 Subclass - Estadão**

In [5]:
class Estadao(DataColector):

    def navega_entre_paginas(self, quanta_vezes):
        lista_geral_de_noticias = []
        for n in range(quanta_vezes):
            html = self.coleta_html()
            codigo_de_noticias = self.coleta_noticia(html)
            lista_de_noticias = self.listar_noticias(codigo_de_noticias)
            lista_geral_de_noticias.extend(lista_de_noticias)
            btn_proxima_pag = self.chrome.find_element_by_xpath('/html/body/section[3]/div/section[1]/div/div/div[2]/div/a[2]')
            self.chrome.execute_script('arguments[0].click();', btn_proxima_pag)
        return lista_geral_de_noticias

    def coleta_noticia(self, fonte):
        return fonte.findAll('section', {'class': 'col-md-12 col-sm-12 col-xs-12 init item-lista'})

    def listar_noticias(self, cod_noticias):
        lista_de_noticias = []
        for noticia in cod_noticias:
            content = {} 
            titulo = noticia.h3
            content['titulo'] = 'None' if not titulo else titulo.text
            resumo = noticia.p
            content['resumo'] = Estadao.coleta_resumo(resumo)
            content['sessao'] = noticia.h4.text
            data = noticia.find('span', {'class': 'data-posts'})
            content['data'] = 'None' if not data else data.text.replace('| ', '')
            content['portal'] = 'Estadão'
            link = noticia.find('a', {'class': 'link-title'})
            content['link'] = 'None' if not link else link['href']
            lista_de_noticias.append(content)
        return lista_de_noticias
      
    @staticmethod
    def coleta_resumo(caminho_para_resumo):
        if not caminho_para_resumo:
            return 'None'
        return caminho_para_resumo.text



if __name__ == '__main__':
    browser = Estadao()
    browser.acessa('https://www.estadao.com.br/ultimas')
    lista_de_noticias_estadao = browser.navega_entre_paginas(6)

    # for news in lista_de_noticias_veja:
    #     for c, v in news.items():
    #         print(f'{c}: {v}')
    #     print('-_' * 40)
    print(len(lista_de_noticias_estadao))
    browser.sair()


150


**2.3 Subclass - Veja**

In [6]:
class Veja(DataColector):
       
    def clica_carregar_mais(self, qnts_vezes):
        try:
            for n in range(qnts_vezes):
                self.aguardar_por(f'//*[@id="infinite-handle"]/span/button')
                element = self.chrome.find_element_by_xpath(f'//*[@id="infinite-handle"]/span/button')
                element.send_keys(Keys.SPACE)
        except Exception as e:
            print(f'Erro ao clicar em carregar mais: {e}')

    def coleta_noticia(self, fonte):
        font = fonte.find('div', {'id': 'infinite-list'})
        return font.findAll('div', {'class': 'row'})
    
    def listar_noticias(self, cod_noticia):
        lista_de_noticias = []
        for news in cod_noticia:
            content = {}
            content['titulo'] = news.h2.text
            resumo = news.find('span', {'class': 'description'}).text
            content['resumo'] = resumo.replace('\t', '').replace('\n', '')
            sessao = news.find('span', {'class': 'category'})
            content['sessao'] = 'None' if not sessao else sessao.text.replace('\n', '', 1)
            data = news.find('span', {'class': 'author'}).find('time', {'class': 'hide-s'})
            content['data'] = 'No date related' if not data else data.text.replace('| ', '')
            content['portal'] = 'Veja'
            content['link'] = news.a['href']
            lista_de_noticias.append(content)
        return lista_de_noticias

if __name__ == '__main__':
    browser = Veja()
    browser.acessa('https://veja.abril.com.br/ultimas-noticias/')
    browser.clica_carregar_mais(10)  
    html = browser.coleta_html()
    browser.sair()
    cod_noticia = browser.coleta_noticia(html)
    lista_de_noticias_veja = browser.listar_noticias(cod_noticia)

    # for noticia in lista_de_noticias_veja:  
    #     for c, v in noticia.items():
    #         print(f'{c}: {v}')
    #     print('¨¨' * 40)
    print(len(lista_de_noticias_veja))

200


**2.3 Subclass - UOL**

In [14]:
class Uol(DataColector):
    
    def clica_ver_mais(self, qnts_vezes):
        try:
            for n in range(qnts_vezes):
                self.scroll()
                self.aguardar_por(f'/html/body/section/div/div/div[1]/section/button')
                element = self.chrome.find_element_by_xpath(f'/html/body/section/div/div/div[1]/section/button')
                element.click()
        except Exception as e:
            print(f'Erro ao clicar em ver mais: {e}')
            
    def coleta_noticia(self, fonte):
        font = fonte.find('div', {'class': 'flex-wrap'})
        return font.findAll('div', {'class': 'thumbnails-wrapper'})

    def listar_noticias(self, cod_noticia):
        lista_de_noticias = []
        for news in cod_noticia:
            content = {}
            titulo = news.h3
            content['titulo'] = 'None' if not titulo else titulo.text
            resumo = news.p
            content['resumo'] = 'None' if not resumo else resumo.text
            sessao = news.span
            content['sessao'] = 'None' if not sessao else sessao.text
            data = news.time
            content['data'] = 'None' if not data else data.text
            content['portal'] = 'UOL'
            content['link'] = news.a['href']
            lista_de_noticias.append(content)
        return lista_de_noticias 

if __name__ == '__main__':
    browser = Uol()
    browser.acessa('https://noticias.uol.com.br/ultimas')
    browser.clica_ver_mais(2)  
    html = browser.coleta_html()
    browser.sair()
    codigo_das_noticias = browser.coleta_noticia(html)
    lista_de_noticias_uol = browser.listar_noticias(codigo_das_noticias)


    # for new in lista_de_noticias_uol:
    #     for c, v in new.items():
    #         print(f'{c}: {v}')
    #     print('¨¨' * 60)
    print(len(lista_de_noticias_uol))

Erro ao clicar em ver mais: Message: 

11


# Main


In [8]:
# Carregando modelos criados e treinados

model = browser.model()
tfidf_vectorize = browser.tfidf_vectorize()

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.linear_model.logistic module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.21.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.21.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py

In [9]:
# Criando dataframe com todas as noticias coletadas

df_g1 = pd.DataFrame(lista_noticias_g1)
df_estadao = pd.DataFrame(lista_de_noticias_estadao)
df_veja = pd.DataFrame(lista_de_noticias_veja)
df_uol = pd.DataFrame(lista_de_noticias_uol)

frames = [df_g1, df_veja, df_uol, df_estadao]
df_news = pd.concat(frames)
df_news.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 547 entries, 0 to 149
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   titulo  547 non-null    object
 1   resumo  539 non-null    object
 2   sessao  544 non-null    object
 3   data    547 non-null    object
 4   portal  547 non-null    object
 5   link    547 non-null    object
dtypes: object(6)
memory usage: 29.9+ KB


In [10]:
# Deletando todos titulos com valor nulo ou 'None'

nulos = df_news[df_news.titulo.isna()].index
df_news = df_news.drop(nulos, axis=0)

for index, row in df_news.iterrows():
    if row['titulo'].lower() == 'none':
        df_news = df_news.drop(index, axis=0)

df_news.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 547 entries, 0 to 149
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   titulo  547 non-null    object
 1   resumo  539 non-null    object
 2   sessao  544 non-null    object
 3   data    547 non-null    object
 4   portal  547 non-null    object
 5   link    547 non-null    object
dtypes: object(6)
memory usage: 29.9+ KB


In [11]:
# Atribuindo sentimento para as noticias com previsão do modelo

matrix_esparsa = tfidf_vectorize.transform(df_news['titulo'])
df_news['sentiment'] = model.predict(matrix_esparsa)
df_news.head()

,titulo,resumo,sessao,data,portal,link,sentiment
0,Anvisa autoriza 3º teste clínico de vacina no ...,Médica é 1ª voluntária a tomar vacina chinesa ...,Coronavírus,2020-07-21 14:17:23.295436,G1,https://g1.globo.com/bemestar/coronavirus/noti...,1
1,Cientista de Oxford diz que há possibilidade d...,GUROVITZ: Vacina em 2021 seria feito comparáve...,Coronavírus,2020-07-21 07:50:23.296092,G1,https://g1.globo.com/bemestar/coronavirus/noti...,0
2,Estado de SP atinge 20 mil mortes com piora da...,"Afastado por problemas no coração, secretário ...",São Paulo,2020-07-21 13:50:23.296725,G1,https://g1.globo.com/sp/sao-paulo/noticia/2020...,1
3,Governo deixará de financiar a maior pesquisa ...,Ministério da Saúde afirma que dará continuida...,Rio Grande do Sul,2020-07-21 07:50:23.297252,G1,https://g1.globo.com/rs/rio-grande-do-sul/noti...,1
4,Cacique da maior tribo indígena do RJ morre co...,Domingos Venite tinha 68 anos. Aldeia em Angra...,Sul do Rio e Costa Verde,2020-07-21 10:50:23.297785,G1,https://g1.globo.com/rj/sul-do-rio-costa-verde...,1


In [12]:
# Atribuindo a probabilidade de sentimento com previsão do modelo

proba_negativa = []
proba_positiva = []
for p in model.predict_proba(matrix_esparsa):
    polarity = p.tolist()
    polarity_neg = polarity[0]
    polarity_pos = polarity[1]
    proba_negativa.append(polarity_neg)
    proba_positiva.append(polarity_pos)

df_news['proba_positiva'] = proba_positiva
df_news['proba_negativa'] = proba_negativa
df_news.head()

,titulo,resumo,sessao,data,portal,link,sentiment,proba_positiva,proba_negativa
0,Anvisa autoriza 3º teste clínico de vacina no ...,Médica é 1ª voluntária a tomar vacina chinesa ...,Coronavírus,2020-07-21 14:17:23.295436,G1,https://g1.globo.com/bemestar/coronavirus/noti...,1,0.598373,0.401627
1,Cientista de Oxford diz que há possibilidade d...,GUROVITZ: Vacina em 2021 seria feito comparáve...,Coronavírus,2020-07-21 07:50:23.296092,G1,https://g1.globo.com/bemestar/coronavirus/noti...,0,0.105270,0.894730
2,Estado de SP atinge 20 mil mortes com piora da...,"Afastado por problemas no coração, secretário ...",São Paulo,2020-07-21 13:50:23.296725,G1,https://g1.globo.com/sp/sao-paulo/noticia/2020...,1,0.794716,0.205284
3,Governo deixará de financiar a maior pesquisa ...,Ministério da Saúde afirma que dará continuida...,Rio Grande do Sul,2020-07-21 07:50:23.297252,G1,https://g1.globo.com/rs/rio-grande-do-sul/noti...,1,0.551765,0.448235
4,Cacique da maior tribo indígena do RJ morre co...,Domingos Venite tinha 68 anos. Aldeia em Angra...,Sul do Rio e Costa Verde,2020-07-21 10:50:23.297785,G1,https://g1.globo.com/rj/sul-do-rio-costa-verde...,1,0.823420,0.176580


In [13]:
model_lr = tfidf_vectorize.transform(df_news['titulo'])

for test, c in zip(df_news['titulo'], model.predict(model_lr)):
    print(test, c)

Anvisa autoriza 3º teste clínico de vacina no Brasil, agora da Pfizer 1
Cientista de Oxford diz que há possibilidade de vacina neste ano 0
Estado de SP atinge 20 mil mortes com piora da situação no interior 1
Governo deixará de financiar a maior pesquisa sobre mapeamento da Covid 1
Cacique da maior tribo indígena do RJ morre com Covid 1
Brasil chega a 80,4 mil mortes por Covid e 2,1 milhões de casos 1
Veja em quais estados as mortes estão subindo e onde estão caindo 1
Governo entrega hoje 1ª parte da reforma tributária, sem 'nova CPMF' 1
GLOBONEWS: assista às últimas notícias e análises no 'Estúdio i' 1
Como será a volta dos cinemas? Vale a pena reabrir agora? 1
Dono da Amazon vê sua fortuna crescer US$ 13 bi em um dia 1
'Você sabe com quem está falando?' e a cultura da carteirada 1
Raoni melhora e pode ter alta até o fim de semana 1
Prima de médica que caiu de prédio na Bahia descarta acidente: 'Ela foi jogada' 1
Ex-capa da Playboy é presa suspeita de tráfico de drogas 1
Empregado é d